### 학습/테스트 데이터 셋 분리 : train_test_split()

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
dt = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt.fit(train_data, train_label)

DecisionTreeClassifier()

**학습 데이터 셋으로 예측을 수행한다면?**

In [3]:
# 학습 데이터 셋으로 예측을 수행한다면?
pred = dt.predict(train_data)
print('예측 정확도:', accuracy_score(train_label, pred))  #당연히 예측 정확도는 1.0

예측 정확도: 1.0


**따라서 train/test data set을 분리한다**

In [4]:
from sklearn.model_selection import train_test_split

dt = DecisionTreeClassifier()
iris_data = load_iris()

# 학습 데이터 세트를 70%, 테스트 데이터 세트를 30%로 분리 
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                   test_size = 0.3, random_state = 10)

# cf. shuffle 인자: default는 shuffle=True / 데이터를 한 번 섞어줄 지 정하는 인자

In [5]:
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
accuracy_score(y_test, pred)

0.9777777777777777

**넘파이 ndarray 뿐만 아니라 판다스 DataFrame/series도 train_test_split() 가능**

In [6]:
import pandas as pd

iris_df = pd.DataFrame(data = iris_data.data, columns=iris_data.feature_names)
iris_df['target'] = iris_data.target
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [7]:
iris_df_x = iris_df.iloc[:, :-1]
iris_df_y = iris_df.iloc[:, -1]

X_train, X_test, Y_train, Y_test = train_test_split(iris_df_x, iris_df_y, test_size = 0.3)

In [8]:
dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)
pred = dt.predict(X_test)
print(accuracy_score(Y_test, pred))

1.0


하지만 붓꽃 데이터의 양 (150개)이 크지 않아 예측 성능을 판단하기에는 적절하지 않다. 

**충분한 학습 데이터 양 보장**과 **다양한 데이터를 활용한 예측 성능 확인**이 중요함!

### Cross Validation

이미 데이터가 적다면...? kfold를 통해 교차 검증을 해보자. 

#### K fold

In [7]:
from sklearn.model_selection import KFold # K-Fold 교차 검증을 위한 라이브러리
import numpy as np

In [8]:
iris = load_iris()
features = iris.data
label = iris.target
classifier = DecisionTreeClassifier(random_state = 10)

In [10]:
# n = 5
# 5개의 폴드 세트로 분리하는 kfold와 폴드 세트별 정확도를 담을 리스트 객체 생성

kfold = KFold(n_splits = 5)
cv_accuracy = []
features.shape   # 150 rows, 4 columns

(150, 4)

In [17]:
kfold.split(features)

<generator object _BaseKFold.split at 0x000001C2A59D59E0>

In [12]:
# KFold 객체의 split( ) 호출 : fold별 train, test data의 row index를 array로 반환
n = 0
for train_index, test_index in kfold.split(features):
      
    # kfold.split( )으로 반환된 인덱스로 train/test data 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # training and prediction
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    n += 1
    
    # print accuracy per every iteration
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n,test_index))
    
    cv_accuracy.append(accuracy)


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.9, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]


In [13]:
np.mean(cv_accuracy)

0.9066599999999999

#### Stratified Kfold

In [19]:
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

위와 같이 label의 분포가 0,1,2 50개씩 있을 때 (심지어 순서대로) k=3인 kfold를 하면?

In [20]:
kfold = KFold(n_splits = 3)
n = 0
for train_idx, test_idx in kfold.split(iris_df):
    label_train = iris_df['label'].iloc[train_idx]
    label_test = iris_df['label'].iloc[test_idx]
    n += 1
    print('교차검증: {0}'.format(n))
    print('train data label:\n', label_train.value_counts())
    print('test data label:\n', label_test.value_counts())

교차검증: 1
train data label:
 1    50
2    50
Name: label, dtype: int64
test data label:
 0    50
Name: label, dtype: int64
교차검증: 2
train data label:
 0    50
2    50
Name: label, dtype: int64
test data label:
 1    50
Name: label, dtype: int64
교차검증: 3
train data label:
 0    50
1    50
Name: label, dtype: int64
test data label:
 2    50
Name: label, dtype: int64


학습 레이블에는 1,2만 또는 0,2만 또는 0,1만 포함되어 나머지 레이블은 전혀 학습의 기회가 없다 -> 해당 레이블에 대한 예측력 : 0

실제 일반화 능력을 정확하게 평가하기 어렵고, 이로 인해 모델의 예측 성능의 신뢰도가 감소할 수 있다. 

**StratifiedKFold는 이렇게 KFold로 분할된 레이블 데이터 세트가 전체 레이블 값의 분포도를 반영하지 못하는 문제를 해결해줌**

In [21]:
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits = 3)
n = 0
for train_idx, test_idx in skfold.split(iris_df, iris_df['label']):

    # 학습 데이터와 검증 데이터의 레이블 추출
    label_train = iris_df['label'].iloc[train_idx]
    label_test = iris_df['label'].iloc[test_idx]
    n += 1

    print('교차검증: {0}'.format(n))
    print('train data label:\n', label_train.value_counts())
    print('test data label:\n', label_test.value_counts())

교차검증: 1
train data label:
 2    34
0    33
1    33
Name: label, dtype: int64
test data label:
 0    17
1    17
2    16
Name: label, dtype: int64
교차검증: 2
train data label:
 1    34
0    33
2    33
Name: label, dtype: int64
test data label:
 0    17
2    17
1    16
Name: label, dtype: int64
교차검증: 3
train data label:
 0    34
1    33
2    33
Name: label, dtype: int64
test data label:
 1    17
2    17
0    16
Name: label, dtype: int64


* 학습레이블과 검증레이블 데이터 값의 분포도가 **거의 동일하게 할당** 되었음을 확인할 수 있다. 

In [18]:
skfold = StratifiedKFold(n_splits=3)
cv_accuracy = []

# StratifiedKFold 객체의 split( ) 호출 : fold별 train, test data의 row index를 array로 반환
n = 0
for train_index, test_index in skfold.split(features, label):
    # skfold.split( )으로 반환된 인덱스로 train/test data 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # training and prediction
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    n += 1
    
    # print accuracy per every iteration
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n,test_index))
    
    cv_accuracy.append(accuracy)

print(np.round(cv_accuracy,4))
print(np.mean(cv_accuracy))


#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.92, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.96, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
[0.98 0.92 0.96]
0.9533333333333333


**결론**

- 왜곡된 레이블 데이터 세트에서는 Stratified K fold를 이용해 교차 검증을 해야 한다
- **회귀 분류** 에서는 stratified k fold를 사용할 수 **없다**

#### cross_val_score()
* scikit learn 에서 교차 검증을 더 편리하게 수행할 수 있게 해주는 API 제공 : cross_val_score()
* cross_val_score(estimator, X, y=None, scoring=None, cv=None)
* estimator : 분류 알고리즘 클래스인 Classifier 또는 회귀 알고리즘 클래스인 Regressor
* X : 피처 데이터 세트
* y : 레이블 데이터 세트
* scoring : 예측 성능 평가 지표
* cv : 교차 검증 폴드 수
* 반환값 : scoring parameter로 지정된 성능 지표 측정값을 배열 형태로 반환

In [22]:
from sklearn.model_selection import cross_val_score, cross_validate

iris_data = load_iris()
classifier = DecisionTreeClassifier(random_state=100)

data = iris_data.data
label = iris_data.target

# 성능지표 = 정확도, 교차검증 세트 = 3개
scores = cross_val_score(classifier, data, y=label, scoring='accuracy', cv=3)

print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.92 0.96]
평균 검증 정확도: 0.9533


cross_val_score의 결과와 stratified k fold의 결과가 똑같다 !!

왜?

- cross_val_score( )가 내부적으로 Stratified K Fold를 이용하기 때문에 결과가 같음을 확인할 수 있음

#### GridSearchCV

교차검증과 최적 하이퍼 파라미터 튜닝을 한 번에!!!


* GridSearchCV(estimator, param_grid, scoring, cv, refit)
* **estimator** : classifier, regressor, pipeline이 사용될 수 있음
* **param_grid** : key + 리스트 값을 가지는 딕셔너리. estimator의 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정함
* **scoring** : 예측 성능을 측정할 평가 방법 지정
* **cv** : 교차 검증을 위해 분할되는 학습/테스트 세트의 개수를 지정
* **refit** : default = 'True', True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킴

In [24]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# loading data and splitting train & test data
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, 
                                                    test_size=0.2, random_state=120)
classifier = DecisionTreeClassifier()

# parameter들을 dictionary 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [25]:
grid_dtree = GridSearchCV(classifier, param_grid=parameters, scoring='accuracy', 
                          cv = 3, refit=True, return_train_score=True)
grid_dtree.fit(X_train, y_train)

#GridSearchCV 결과는 cv_results_라는 딕셔너리로 저장됨. 이를 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.000668,0.000473,0.000997,5.150430e-07,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.675,0.675,0.650,0.666667,0.011785,5,0.6625,0.6625,0.6750,0.666667,0.005893
1,0.000665,0.000470,0.000332,4.699093e-04,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.675,0.675,0.650,0.666667,0.011785,5,0.6625,0.6625,0.6750,0.666667,0.005893
2,0.001152,0.000219,0.000502,4.072768e-04,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.950,0.900,0.875,0.908333,0.031180,3,0.9500,0.9750,0.9625,0.962500,0.010206
3,0.000333,0.000470,0.000665,4.700217e-04,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.950,0.900,0.875,0.908333,0.031180,3,0.9500,0.9750,0.9625,0.962500,0.010206
4,0.001001,0.000005,0.000716,5.099904e-04,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.950,0.875,0.975,0.933333,0.042492,1,0.9500,0.9875,0.9750,0.970833,0.015590
5,0.000838,0.000207,0.000688,4.863113e-04,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.950,0.875,0.975,0.933333,0.042492,1,0.9500,0.9875,0.9750,0.970833,0.015590


In [26]:
# GridSearchCV 최적 파라미터
print(grid_dtree.best_params_)

# GridSearchCV 최고 정확도 ("mean_test_score" column 확인하기)
print(grid_dtree.best_score_)

# refit=True로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된 Estimator를 내포하고 있으므로 predict()를 통해 예측도 가능. 
pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

{'max_depth': 3, 'min_samples_split': 2}
0.9333333333333332
테스트 데이터 세트 정확도: 1.0000


In [24]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 1.0000
